# Lab 2.4.3: MoE Exploration

**Module:** 2.4 - Efficient Architectures  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this lab, you will:
- [ ] Understand the Mixture of Experts (MoE) architecture
- [ ] Load and run MoE models on DGX Spark
- [ ] Analyze expert activation patterns across different prompts
- [ ] Compare total vs active parameters

---

## 📚 Prerequisites

- Completed: Lab 2.4.1-2.4.2 (Mamba labs)
- Knowledge of: Transformer architecture, feed-forward layers
- Hardware: DGX Spark (128GB) for DeepSeekMoE-16B, or 24GB+ GPU for smaller models

---

## 🌍 Real-World Context

**The Scaling Challenge**

Everyone wants larger models (they perform better!), but:
- GPT-4: ~1.7 trillion parameters (rumored)
- Cost: Millions of dollars to train and run
- Inference: Slow and expensive

**MoE's solution**: Have 1 trillion parameters, but only USE 100 billion per token!

**Real-world MoE models:**
- **Mixtral 8x7B**: 45B total, 12.9B active (3.5x efficiency)
- **DeepSeekMoE-16B**: 16B total, 2.5B active (6.4x efficiency!)
- **GPT-4**: Rumored to be MoE (8 experts, ~200B each)
- **DeepSeek-V3**: 671B total, 37B active (18x efficiency)

On DGX Spark, you can run MoE models that would require a data center with dense architectures!

---

## 🧒 ELI5: Mixture of Experts

> **Imagine a hospital with specialist doctors...**
>
> **Dense model approach**: One super-doctor who knows everything about every medical specialty. They're amazing but:
> - Very expensive to train (medical school for 50 years!)
> - Takes forever for each consultation (thinking about everything)
>
> **MoE approach**: A hospital with many specialists:
> - Cardiologist (heart expert)
> - Neurologist (brain expert)  
> - Dermatologist (skin expert)
> - ... and a smart receptionist (router)
>
> When you arrive, the receptionist looks at your symptoms and sends you to the RIGHT specialists (usually 2-3). You don't see ALL doctors—just the relevant ones!
>
> **In AI terms:**
> - Each "expert" is a feed-forward network (the expensive part of transformers)
> - The "router" is a small network that picks which experts to use
> - For each token, only top-k experts are activated (k=2 typically)
> - Result: 10x parameters, but only 1x compute!

### The Key Insight: Sparse Activation

```
Dense model:  All parameters used for every token
              Total params = Active params = 70B
              
MoE model:    Many parameters, few used per token
              Total params = 200B
              Active params = 20B (only 10%!)
              
Same compute cost, but 10x more knowledge stored!
```

---

## Part 1: Setup

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import Dict, List, Tuple, Optional
import gc
import time

# Check environment
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"Memory: {total_memory:.1f} GB")
    
    if total_memory > 100:
        print("\n🚀 DGX Spark detected! You can run large MoE models.")
        RECOMMENDED_MODEL = "deepseek-ai/deepseek-moe-16b-base"
    elif total_memory > 20:
        print("\n💡 Large GPU detected. Some MoE models will fit.")
        RECOMMENDED_MODEL = "Qwen/Qwen1.5-MoE-A2.7B"
    else:
        print("\n⚠️ Limited GPU memory. Using 8-bit quantization.")
        RECOMMENDED_MODEL = "Qwen/Qwen1.5-MoE-A2.7B"
else:
    print("⚠️ No CUDA available. This lab requires GPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

def clear_gpu():
    """Clear GPU memory."""
    torch.cuda.empty_cache()
    gc.collect()
    if torch.cuda.is_available():
        print(f"GPU memory: {torch.cuda.memory_allocated()/1e9:.2f} GB allocated")

---

## Part 2: Loading an MoE Model

### Available MoE Models

| Model | Total Params | Active Params | Memory (FP16) | DGX Spark? |
|-------|-------------|---------------|---------------|------------|
| Qwen1.5-MoE-A2.7B | 14.3B | 2.7B | ~29GB | ✅ Easy |
| DeepSeekMoE-16B | 16B | 2.5B | ~32GB | ✅ Easy |
| Mixtral-8x7B | 45B | 12.9B | ~90GB | ✅ Fits |
| DeepSeek-V3 | 671B | 37B | ~1.3TB | ❌ Too large |

In [ ]:
# Choose model based on available memory
# Uncomment the model you want to use

# MODEL_NAME = "Qwen/Qwen1.5-MoE-A2.7B"  # Smaller, fits on 24GB
MODEL_NAME = "deepseek-ai/deepseek-moe-16b-base"  # Recommended for DGX Spark
# MODEL_NAME = "mistralai/Mixtral-8x7B-v0.1"  # Larger, needs ~90GB

print(f"Loading {MODEL_NAME}...")
print("(This may take a few minutes for large models)\n")

clear_gpu()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
# For limited memory, add: load_in_8bit=True or load_in_4bit=True
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Report stats
total_params = sum(p.numel() for p in model.parameters())
memory_used = torch.cuda.memory_allocated() / 1e9

print(f"\n✅ Model loaded!")
print(f"   Total parameters: {total_params/1e9:.2f}B")
print(f"   GPU Memory used: {memory_used:.2f} GB")

In [ ]:
# Extract MoE configuration
config = model.config

# Different models store this differently
try:
    num_experts = getattr(config, 'num_experts', 
                         getattr(config, 'n_routed_experts',
                                getattr(config, 'num_local_experts', 8)))
    experts_per_token = getattr(config, 'num_experts_per_tok',
                               getattr(config, 'top_k', 2))
except:
    num_experts = 8
    experts_per_token = 2

print(f"\n📊 MoE Architecture:")
print(f"   Number of experts: {num_experts}")
print(f"   Experts per token: {experts_per_token}")
print(f"   Sparsity ratio: {num_experts / experts_per_token:.1f}x")
print(f"   Active params (approx): {total_params / (num_experts / experts_per_token) / 1e9:.2f}B")

### 🔍 What's Special About MoE Memory?

Notice that we loaded ALL parameters (~32GB for DeepSeekMoE-16B), but:
- During inference, only ~2.5B parameters are actually **computed**
- This means 6x+ compute savings compared to a dense 16B model
- On DGX Spark's 128GB, we have tons of room for batch processing!

---

## Part 3: Testing MoE Inference

In [ ]:
def generate_text(prompt: str, max_tokens: int = 50) -> str:
    """Generate text with the MoE model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with different types of prompts
test_prompts = {
    "Code": "def fibonacci(n):",
    "Math": "The derivative of x^2 is",
    "Story": "Once upon a time,",
    "Science": "Photosynthesis is the process",
}

print("Testing MoE generation on different domains:\n")
print("=" * 60)

for domain, prompt in test_prompts.items():
    print(f"\n📝 {domain}:")
    print(f"Prompt: {prompt}")
    result = generate_text(prompt, max_tokens=30)
    print(f"Output: {result}")
    print("-" * 40)

---

## Part 4: Analyzing Expert Activation Patterns

The key question: **Do different experts specialize in different topics?**

Let's hook into the router to see which experts activate for different inputs.

### PyTorch Hooks: Inspecting Model Internals

PyTorch provides **hooks** to intercept data flowing through a model. We'll use `register_forward_hook` to capture router outputs:

```python
def hook_function(module, input, output):
    """Called every time the module processes data."""
    # module: The layer being hooked
    # input: Tuple of inputs to the layer
    # output: The layer's output
    captured_outputs.append(output)

# Attach hook to a module
hook_handle = layer.register_forward_hook(hook_function)

# Run inference - hook captures outputs
model(input_data)

# Remove hook when done
hook_handle.remove()
```

This is powerful for:
- Visualizing intermediate activations
- Debugging model behavior
- Analyzing which experts activate in MoE models

In [ ]:
class ExpertActivationTracker:
    """
    Track which experts are activated during inference.
    
    Uses hooks to capture router outputs.
    """
    
    def __init__(self, model, num_experts: int = 64):
        self.model = model
        self.num_experts = num_experts
        self.activations = []  # Store (layer, token_idx, expert_indices, weights)
        self.hooks = []
        
    def _find_router_modules(self):
        """Find router/gate modules in the model."""
        routers = []
        
        for name, module in self.model.named_modules():
            # Different models use different names
            if 'gate' in name.lower() or 'router' in name.lower():
                if hasattr(module, 'weight'):  # Linear layer
                    routers.append((name, module))
        
        return routers
    
    def _hook_fn(self, layer_idx):
        """Create a hook function for a specific layer."""
        def hook(module, input, output):
            # Output is typically router logits [batch, seq, num_experts]
            if isinstance(output, tuple):
                router_logits = output[0]
            else:
                router_logits = output
            
            if router_logits is not None and len(router_logits.shape) >= 2:
                self.activations.append({
                    'layer': layer_idx,
                    'logits': router_logits.detach().cpu(),
                })
        return hook
    
    def attach_hooks(self):
        """Attach hooks to router modules."""
        routers = self._find_router_modules()
        print(f"Found {len(routers)} router modules")
        
        for idx, (name, module) in enumerate(routers[:8]):  # First 8 layers
            hook = module.register_forward_hook(self._hook_fn(idx))
            self.hooks.append(hook)
    
    def remove_hooks(self):
        """Remove all hooks."""
        for hook in self.hooks:
            hook.remove()
        self.hooks = []
    
    def clear(self):
        """Clear stored activations."""
        self.activations = []
    
    def get_expert_counts(self, top_k: int = 2) -> Dict[int, int]:
        """Count how often each expert was in top-k."""
        counts = defaultdict(int)
        
        for act in self.activations:
            logits = act['logits']
            if len(logits.shape) == 3:  # [batch, seq, experts]
                for batch in range(logits.shape[0]):
                    for seq in range(logits.shape[1]):
                        top_experts = torch.topk(logits[batch, seq], k=top_k).indices
                        for exp in top_experts.tolist():
                            counts[exp] += 1
            elif len(logits.shape) == 2:  # [tokens, experts]
                for seq in range(logits.shape[0]):
                    top_experts = torch.topk(logits[seq], k=top_k).indices
                    for exp in top_experts.tolist():
                        counts[exp] += 1
        
        return dict(counts)

# Create tracker
tracker = ExpertActivationTracker(model, num_experts=num_experts)
print("Created ExpertActivationTracker")

In [ ]:
# Attach hooks and run inference
tracker.attach_hooks()

# Define test prompts for different domains
domain_prompts = {
    "Code": [
        "def quicksort(arr):",
        "import numpy as np\nimport pandas as pd",
        "class DatabaseConnection:",
        "SELECT * FROM users WHERE",
    ],
    "Math": [
        "The integral of sin(x) is",
        "Solve the equation: 2x + 5 = 15",
        "The Pythagorean theorem states",
        "Calculate the derivative of",
    ],
    "Creative": [
        "Once upon a time in a magical kingdom",
        "The sunset painted the sky with",
        "She walked through the ancient forest",
        "The old lighthouse keeper watched",
    ],
    "Science": [
        "The theory of relativity explains",
        "DNA replication begins when",
        "The laws of thermodynamics state",
        "Quantum entanglement is a phenomenon",
    ],
}

# Analyze expert activation for each domain
domain_expert_counts = {}

for domain, prompts in domain_prompts.items():
    print(f"\nAnalyzing {domain} prompts...")
    tracker.clear()
    
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            _ = model(**inputs)
    
    counts = tracker.get_expert_counts(top_k=experts_per_token)
    domain_expert_counts[domain] = counts
    
    # Show top 5 experts for this domain
    sorted_experts = sorted(counts.items(), key=lambda x: -x[1])[:5]
    print(f"  Top experts: {sorted_experts}")

tracker.remove_hooks()
print("\n✅ Expert analysis complete!")

In [ ]:
# Visualize expert specialization

# Normalize counts for each domain
all_experts = set()
for counts in domain_expert_counts.values():
    all_experts.update(counts.keys())
all_experts = sorted(all_experts)

# Create matrix: domains x experts
matrix = np.zeros((len(domain_prompts), len(all_experts)))
domain_names = list(domain_prompts.keys())

for i, domain in enumerate(domain_names):
    counts = domain_expert_counts[domain]
    total = sum(counts.values()) + 1e-8
    for j, exp in enumerate(all_experts):
        matrix[i, j] = counts.get(exp, 0) / total

# Plot heatmap
fig, ax = plt.subplots(figsize=(14, 6))
im = ax.imshow(matrix, aspect='auto', cmap='YlOrRd')

ax.set_yticks(range(len(domain_names)))
ax.set_yticklabels(domain_names, fontsize=12)
ax.set_xlabel('Expert Index', fontsize=12)
ax.set_ylabel('Domain', fontsize=12)
ax.set_title('Expert Activation Patterns by Domain\n(Darker = More Active)', 
             fontsize=14, fontweight='bold')

# Show expert indices for top-activated ones
if len(all_experts) <= 30:
    ax.set_xticks(range(len(all_experts)))
    ax.set_xticklabels(all_experts, rotation=45, fontsize=8)

plt.colorbar(im, ax=ax, label='Activation Frequency')
plt.tight_layout()
plt.show()

# Find domain-specific experts
print("\n🔍 Domain-Specific Expert Analysis:")
print("=" * 50)

for i, domain in enumerate(domain_names):
    # Find experts that are most activated for this domain vs others
    domain_activations = matrix[i]
    other_activations = np.mean(matrix[[j for j in range(len(domain_names)) if j != i]], axis=0)
    
    specialization = domain_activations - other_activations
    top_specialized = np.argsort(specialization)[-3:][::-1]
    
    print(f"\n{domain}:")
    for exp_idx in top_specialized:
        exp_id = all_experts[exp_idx]
        score = specialization[exp_idx]
        if score > 0:
            print(f"  Expert {exp_id}: +{score:.3f} specialization")

### 🔍 What Do Expert Patterns Tell Us?

The visualization shows:
1. **Some experts are "generalists"** - activated frequently across all domains
2. **Some experts are "specialists"** - activated more for specific domains
3. **The model learned this naturally** - no explicit domain labels during training!

This emergent specialization is what makes MoE powerful.

---

## Part 5: Memory vs Compute Analysis

In [ ]:
# Analyze memory vs compute efficiency

def analyze_moe_efficiency(total_params: int, active_params: int, 
                           dense_equivalent_params: int) -> Dict:
    """
    Analyze MoE efficiency compared to dense models.
    """
    return {
        "total_params_B": total_params / 1e9,
        "active_params_B": active_params / 1e9,
        "sparsity_ratio": total_params / active_params,
        "compute_savings": (1 - active_params / total_params) * 100,
        "memory_fp16_GB": total_params * 2 / 1e9,
        "memory_fp8_GB": total_params * 1 / 1e9,
        "memory_fp4_GB": total_params * 0.5 / 1e9,
        "equivalent_dense_memory_GB": dense_equivalent_params * 2 / 1e9,
    }

# Compare different MoE models
moe_models = {
    "DeepSeekMoE-16B": (16e9, 2.5e9, 2.5e9),  # (total, active, dense_equiv)
    "Mixtral-8x7B": (45e9, 12.9e9, 12.9e9),
    "Qwen-MoE-A2.7B": (14.3e9, 2.7e9, 2.7e9),
    "GPT-4 (rumored)": (1700e9, 200e9, 200e9),
}

print("📊 MoE Model Efficiency Analysis")
print("=" * 80)
print(f"{'Model':<20} {'Total':<10} {'Active':<10} {'Sparsity':<10} {'Memory (FP16)':<15}")
print("-" * 80)

for name, (total, active, dense) in moe_models.items():
    analysis = analyze_moe_efficiency(total, active, dense)
    print(f"{name:<20} {analysis['total_params_B']:.1f}B      "
          f"{analysis['active_params_B']:.1f}B      "
          f"{analysis['sparsity_ratio']:.1f}x       "
          f"{analysis['memory_fp16_GB']:.1f} GB")

print("\n💡 Key Insight: MoE lets you have much more 'knowledge' (total params)")
print("   while keeping inference cost manageable (active params)!")

In [ ]:
# Visualize the efficiency tradeoff

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Data for visualization
models = list(moe_models.keys())
totals = [moe_models[m][0]/1e9 for m in models]
actives = [moe_models[m][1]/1e9 for m in models]

# Left: Bar chart of total vs active
x = np.arange(len(models))
width = 0.35

bars1 = axes[0].bar(x - width/2, totals, width, label='Total Params', color='#3498DB')
bars2 = axes[0].bar(x + width/2, actives, width, label='Active Params', color='#27AE60')

axes[0].set_xlabel('Model')
axes[0].set_ylabel('Parameters (Billions)')
axes[0].set_title('Total vs Active Parameters', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models, rotation=45, ha='right')
axes[0].legend()
axes[0].set_yscale('log')

# Right: Efficiency ratio
ratios = [t/a for t, a in zip(totals, actives)]
colors = plt.cm.RdYlGn([r/max(ratios) for r in ratios])

bars = axes[1].bar(models, ratios, color=colors)
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Efficiency Ratio (Total/Active)')
axes[1].set_title('MoE Efficiency Ratio\n(Higher = More Efficient)', fontweight='bold')
axes[1].set_xticklabels(models, rotation=45, ha='right')

# Add value labels
for bar, ratio in zip(bars, ratios):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                f'{ratio:.1f}x', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n🚀 On DGX Spark (128GB):")
for name, (total, active, _) in moe_models.items():
    memory_needed = total * 2 / 1e9  # FP16
    if memory_needed < 128:
        print(f"   ✅ {name}: {memory_needed:.0f}GB fits!")
    else:
        print(f"   ❌ {name}: {memory_needed:.0f}GB too large")

---

## ⚠️ Common Mistakes

### Mistake 1: Thinking All Experts Are Always Loaded
```python
# ✅ All experts ARE in memory (needed for routing)
# ❌ But NOT all are computed per token!

# Memory: Total params × precision
# Compute: Active params × precision
```

### Mistake 2: Ignoring Load Balancing
```python
# ❌ Some experts may be overused, others ignored
# This wastes capacity!

# ✅ MoE models include auxiliary loss for load balancing
# Check if your model has balanced expert usage
```

### Mistake 3: Wrong Batch Size Expectations
```python
# ❌ "MoE should be fast with any batch size"
# ✅ MoE benefits from batching - experts can process multiple tokens

# For efficiency, use batch_size >= num_experts_per_token
```

---

## 🎉 Checkpoint

You've learned:
- ✅ How MoE achieves efficiency through sparse activation
- ✅ Loading and running MoE models on DGX Spark
- ✅ Expert specialization emerges from training
- ✅ Total vs active parameter tradeoffs
- ✅ Visualizing expert activation patterns

---

## ✋ Try It Yourself

### Exercise: Custom Domain Analysis
Add new domain categories and analyze expert activation:
1. Legal text
2. Medical text
3. Financial text

Which experts specialize in these domains?

In [ ]:
# Your code here



---

## 📖 Further Reading

- [Mixtral Paper](https://arxiv.org/abs/2401.04088) - Mixture of Experts Made Easy
- [DeepSeekMoE Paper](https://arxiv.org/abs/2401.06066) - Efficient MoE Architecture
- [Switch Transformer](https://arxiv.org/abs/2101.03961) - Simplified MoE at Scale
- [GShard Paper](https://arxiv.org/abs/2006.16668) - Scaling Giant Models with MoE

---

## 🧹 Cleanup

In [ ]:
# Cleanup
if 'model' in dir():
    del model
if 'tokenizer' in dir():
    del tokenizer
if 'tracker' in dir():
    tracker.remove_hooks()
    del tracker

torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")